# Caso: Tweets Geolocalizados
**Descripción:** Análisis de tweets según ubicación y horario.

**Objetivo:** Detectar zonas de influencia política.

**Diccionario de datos:**
- `latitude`: Latitud del tweet.
- `longitude`: Longitud del tweet.
- `tweet_time`: Minuto del día en que se publicó el tweet.

## Paso 1: Carga de datos

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN, MeanShift, estimate_bandwidth
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar dataset
df = pd.read_csv("/mnt/data/datasets_clustering_reales/Tweets_Geolocalizados.csv")
df.head()

## Paso 2: Preprocesamiento

In [ ]:
# Normalización
scaler = StandardScaler()
X = scaler.fit_transform(df)

## Paso 3: Clustering con DBSCAN

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=5)
labels_dbscan = dbscan.fit_predict(X)
df['DBSCAN'] = labels_dbscan

## Paso 4: Clustering con Mean-Shift

In [ ]:
bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=1000)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
labels_ms = ms.fit_predict(X)
df['MeanShift'] = labels_ms

## Paso 5: Visualización de Resultados

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.scatterplot(x=df.iloc[:,0], y=df.iloc[:,1], hue=df['DBSCAN'], palette='tab10')
plt.title("DBSCAN")

plt.subplot(1,2,2)
sns.scatterplot(x=df.iloc[:,0], y=df.iloc[:,1], hue=df['MeanShift'], palette='tab10')
plt.title("Mean-Shift")
plt.show()

## Paso 6: Evaluación con Silhouette Score

In [ ]:
def safe_silhouette(X, labels):
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    return silhouette_score(X, labels) if n_clusters > 1 else -1

score_dbscan = safe_silhouette(X, df['DBSCAN'])
score_meanshift = safe_silhouette(X, df['MeanShift'])

print(f"Silhouette DBSCAN: {score_dbscan:.4f}")
print(f"Silhouette Mean-Shift: {score_meanshift:.4f}")

## Paso 7: Conclusiones

In [ ]:
if score_dbscan > score_meanshift:
    print("DBSCAN obtuvo un mejor desempeño en este caso.")
elif score_meanshift > score_dbscan:
    print("Mean-Shift obtuvo un mejor desempeño en este caso.")
else:
    print("Ambos algoritmos obtuvieron resultados similares o no fueron efectivos.")